## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hanninga (`np.hanning`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:

        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate

lena = cv2.imread("lena.bmp", cv2.IMREAD_GRAYSCALE)

def fourier2(image):
    image = image.astype('float32')
    F_fcn = cv2.dft(image, flags = cv2.DFT_COMPLEX_OUTPUT)
    F_fcn = np.fft.fftshift(F_fcn, [0, 1])
    magnitude, phase = cv2.cartToPolar(F_fcn[:, :, 0], F_fcn[:, :, 1])
    magnitude_log = np.log10(magnitude + 1)
    return F_fcn, magnitude_log, phase

def inverse_fourier_transform(image):
    fft_shifted = np.fft.ifftshift(image, [0, 1])
    ifft = cv2.idft(fft_shifted, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    imgIFFT = cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])
    reconstructed_image = np.round(imgIFFT).astype(np.uint8)
    return reconstructed_image

def plot_3(image1, image2, image3, suptitle):
    fig, (ax1,ax2,ax3) = plt.subplots(1, 3,figsize=(20,6))
    fig.suptitle("image: " + suptitle)
    ax1.imshow(image1, 'gray')
    ax1.set_title('Original')
    ax1.axis('off')

    ax2.imshow(image2, 'gray')
    ax2.set_title('Filtered IFFT')
    ax2.axis('off')
    
    ax3.imshow(image3, 'gray')
    ax3.set_title('After subtraction')
    ax3.axis('off')
    plt.show()

In [ ]:
filter_size = 21

window = np.hamming(filter_size)
window2d = np.outer(window, window)

window_1 = [filter_size, filter_size]
FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(window_1[0]))
FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, window_1[1]]))
FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(window_1[1]))
FSpaceColsM = np.outer(np.ones([1, window_1[0]]), FSpaceCols)

FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

FilterF = FreqR >= 0.2

FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
FilterFidft = cv2.idft(np.float32(FilterFRot3), flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

filtr_l = window2d * FilterFI

filtr_padd = cv2.copyMakeBorder(filtr_l, 246, 245, 246, 245, cv2.BORDER_CONSTANT, value=0)
filtr_padd = filtr_padd.astype("float32")
F_fcn_filtr = cv2.dft(filtr_padd, flags=cv2.DFT_COMPLEX_OUTPUT)
F_fcn_filtr = np.fft.fftshift(F_fcn_filtr, [0, 1])
F_fcn_filtr_abs = np.abs(F_fcn_filtr)

size = lena.shape
FSpaceRows_1 = 2 * np.fft.fftshift(np.fft.fftfreq(size[0]))
FSpaceRowsM_1 = np.outer(FSpaceRows_1, np.ones([1, size[1]]))
FSpaceCols_1 = 2 * np.fft.fftshift(np.fft.fftfreq(size[1]))
FSpaceColsM_1 = np.outer(np.ones([1, size[0]]), FSpaceCols_1)  

figFilter = plt.figure()
axsFilter = figFilter.add_subplot(projection='3d')
axsFilter.plot_surface(FSpaceRowsM_1, FSpaceColsM_1, F_fcn_filtr_abs[:,:,0], rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)

lena_fft, magnitude_log, phase = fourier2(lena)
lena_filtr = lena_fft * F_fcn_filtr_abs
lena_inv = inverse_fourier_transform(lena_filtr)
diff = np.abs(lena - lena_inv)

plot_3(lena, lena_inv, diff, "Lena")